In [52]:
import pandas as pd
import numpy as np
import requests

In [61]:
#api_key = 'Please insert your API KEY here'

In [54]:
 def destinations_info(origin,destination):
    url = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={origin}&destinations={destination}&units=kilometers&key={api_key}"
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
        
    distance = (response_data)["rows"][0]["elements"][0]["distance"]["text"]
    target = (response_data)["destination_addresses"][0].split(',')[0]
    duration = (response_data)["rows"][0]["elements"][0]["duration"]["value"]
    hours,minutes = duration // 3600 , (duration % 3600) // 60
    duration= f'{hours} hours and {minutes} minutes'
    
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (destination,api_key)
        
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    
    lng = response_data ['results'][0]["geometry"]["location"]['lng']    
    lat = response_data ['results'][0]["geometry"]["location"]['lat']
    
    return target,distance,duration,lng,lat

In [55]:
# Create an empty dictionary to hold the values
my_dict = {}

# Create empty lists to hold the values for each column
target_lst ,dist_lst ,Dur_lst ,long_lst ,lat_lst = [],[],[],[],[]

# Open the file containing the destinations
fname = open('dests.txt')

origin = 'Tel Aviv'

# Read each line from the file and get the destinations
for line in fname:
    line = line.rstrip()
    target, distance,duration,lng, lat = destinations_info(origin, line)
    target_lst.append(target)
    dist_lst.append(distance)
    Dur_lst.append(duration)
    long_lst.append(lng)
    lat_lst.append(lat)

# Add the lists to the dictionary
my_dict['Target'] = target_lst
my_dict['Distance_km'] = dist_lst
my_dict['Duration (hour+minutes)'] = Dur_lst
my_dict['Longitude'] = long_lst
my_dict['latitude'] = lat_lst

# Convert the dictionary to a pandas dataframe
df = pd.DataFrame.from_dict(my_dict)

In [56]:
df

,Target,Distance_km,Duration (hour+minutes),Longitude,latitude
0,İstanbul,"1,815 km",21 hours and 2 minutes,28.978359,41.008238
1,Amsterdam,"4,533 km",48 hours and 3 minutes,4.904139,52.367573
2,Valletta,"3,793 km",50 hours and 50 minutes,14.514100,35.899237
3,Basel,"4,093 km",44 hours and 2 minutes,7.588576,47.559599
4,Doha,"2,164 km",22 hours and 38 minutes,51.531040,25.285447


In [57]:
df_copy = df.copy()
# extract the distance values as floats
df_copy['Distance_km'] = df_copy['Distance_km'].str.replace(',', '').str.replace(' km', '').astype(float)

# sort the dataframe by distance in descending order
df_copy = df_copy.sort_values(by='Distance_km', ascending=False)

# print the top 3 cities with the longest distance
top_cities = df_copy.head(3)['Target'].tolist()
print("The 3 cities with the longest distance are:", top_cities[0]+', '+top_cities[1]+' and '+top_cities[2])

The 3 cities with the longest distance are: Amsterdam, Basel and Valletta


In [58]:
df_copy.head(3)

,Target,Distance_km,Duration (hour+minutes),Longitude,latitude
1,Amsterdam,4533.0,48 hours and 3 minutes,4.904139,52.367573
3,Basel,4093.0,44 hours and 2 minutes,7.588576,47.559599
2,Valletta,3793.0,50 hours and 50 minutes,14.514100,35.899237
